In [ ]:
# -*- coding: utf-8 -*-
"""Insurance_Claim.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1Y_rHaNtM9EkVlPbyOpaGnHmO0FpBwchH

###**Detecting Insurance Claim Fraud with Machine Learning**

This project aims to detect fraudulent insurance claims using the ‘insurance_claims.csv’ dataset. It involves data preprocessing, feature selection, and applying supervised classification algorithms to predict fraud likelihood, improving fraud detection efficiency for insurance providers.
"""

# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report # Import confusion_matrix and classification_report
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate, StratifiedKFold
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

#Load the Dataset File
insurance_df = pd.read_csv('insurance_claims.csv')
insurance_df.head()

"""####**Data Preprocessing**"""

# View features, their data types and count of null values
insurance_df.info()

"""**Data Information Summary:**


The dataset consists of 1,000 customer records with 38 features and one target variable. The features encompass customer details, policy information, incident specifics, and a fraud indicator.

Upon inspection, the ‘authorities_contacted’ column contains 91 missing values, while the ’_c39’ column appears to have no values and can be safely dropped.

The other columns did not have any errors or missing values.
"""

# Drop c_39 column
insurance_df.drop(['_c39'], axis=1, inplace=True)

"""**Handling Null values in 'Authorities Contacted':**


Upon closer inspection, the ‘Authorities Contacted’ column does not have actual null values; instead, the value “None” was misinterpreted as NaN. To correct this, NaN values were manually replaced with “None” to retain accurate data representation.
"""

#Replace NaN with "None" in authorities contacted column
insurance_df.fillna({'authorities_contacted':'None'}, inplace=True)
null_count = insurance_df['authorities_contacted'].isna().sum()
print(f"Number of null values in 'Authorities Contacted': {null_count}")

# Summary of numerical columns in data
insurance_df.describe()

"""####**Exploratory Data Analysis**

**Feature Distribution**
"""

# Drop numerical columns that do not provide meaningful business insights when plotted as distributions
df_filtered = insurance_df.drop(columns=['policy_number', 'insured_zip', 'witnesses'])

# Plot histogram for all numerical columns
df_filtered.hist(figsize=(20,20))
plt.show()

"""General Insights from Histograms of Insurance Claims Data:

1. Claim Amount:


Right-skewed Distribution: The histogram of claim amounts is right-skewed, indicating that most claims are for smaller amounts, with a few larger claims driving the tail of the distribution.



2. Age:

Roughly Normal or Bimodal Distribution: The histogram of insured age is observed to be a bimodal distribution which indicates distinct customer segments (e.g., younger and older drivers).

Impact on Risk: Age is an important factor in assessing risk, as younger and older drivers may have higher accident rates compared to middle-aged drivers.


3. Policy Deductible & Policy Annual Premium:

Policy deductible is $500, $1,000 or $2,000.

The policy annual premium appears to be normally distributed with a mean of about $1,256.


4. Umbrella Limit, Capital Gains & Capital Loss:

A significant proportion of customers have an umbrella limit, capital gain, or capital loss of $0, indicating that these financial protections or transactions are either uncommon or not utilized by most policyholders. This trend could suggest a lower prevalence of high-net-worth individuals in the dataset or a general preference for minimal risk coverage.

5. Claim Distribution:

Total Claim is the sum of vehicle claim, injury claim, and property claim. The distribution of claims is right-skewed, with a large number of customers having no claims, while the number of customers decreases as claim amounts increase. Notably, there is a spike in vehicle claims between $4000

and

$5000, which also influences the overall claim distribution since vehicle claims make up the largest portion of total claims.

**Correlation Heatmap**
"""

#select numerical columns
numerical_columns = insurance_df.select_dtypes(include=['int64','float64']).columns
#correlation matrix
correlation_matrix = insurance_df[numerical_columns].corr()
plt.figure(figsize=(25,25))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

"""The correlation matrix shows a high correlation between total_claim_amount and injury_claim, property_claim, and vehicle_claim, as total_claim_amount is simply their sum. Given this redundancy, the total_claim_amount column can be dropped to avoid multicollinearity.

**Scatterplot**
"""

sns.pairplot(insurance_df, kind="scatter", diag_kind=None)
plt.figure(figsize=(40,40))
plt.show()

"""**Boxplot Analysis for Outlier Detection**"""

# Outlier Detection using Boxplots
# Calculate the number of rows needed for the subplots
num_rows = (len(numerical_columns) + 2) // 3

# Adjust figure size to accommodate all subplots and prevent overlap
plt.figure(figsize=(15, 5 * num_rows))

# Create boxplots for each column in subplots
for i, col in enumerate(numerical_columns):
    plt.subplot(num_rows, 3, i + 1)
    # Using plt.boxplot to create the boxplot for the column
    plt.boxplot(insurance_df[col])
    plt.title(col)
    plt.xticks(rotation=45)  # Rotate x-axis labels

plt.suptitle("Outlier Detection using Boxplots", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

"""The majority of data points fall within the interquartile range (IQR), suggesting that outliers have minimal impact on the dataset.

**Summary of Categorical Variables**
"""

#Extract categorical variable columns
categorical_columns = insurance_df.select_dtypes(include=['object']).columns

for col in categorical_columns:
    print(f"Feature: {col}")
    print(insurance_df[col].value_counts())
    print("-" * 20)

"""**Class Imbalance**"""

#Calculate class imbalance
insurance_df['fraud_reported'].value_counts()

"""The count of records reported as fraud indicate there is class imbalance. This needs to be addressed during classification through class imbalance handling techniques.

####**Model Building**

**Feature Selection**
"""

#Remove unnecessary columns to eliminate irrelevant or redundant data, reducing overfitting, and speeding up the computation process.
insurance_df.drop(['age', 'policy_bind_date', 'policy_number','policy_csl','policy_deductable','insured_relationship','incident_hour_of_the_day','umbrella_limit', 'insured_zip','insured_hobbies', 'capital-gains', 'capital-loss', 'incident_city', 'incident_date', 'incident_location', 'witnesses','number_of_vehicles_involved', 'auto_make', 'auto_year'], axis=1, inplace=True)
insurance_df.head()

"""**Encoding Categorical Variables**"""

# Extract columns with categorical variables from updated data
categorical_columns = insurance_df.select_dtypes(include=['object']).columns

# Perform Label Encoding to covert text to numerical values
label_encoder = LabelEncoder()
for col in categorical_columns:
    insurance_df[col] = label_encoder.fit_transform(insurance_df[col])

insurance_df.head()

"""**Split model into train and test set**"""

#split model into train and test
X = insurance_df.drop('fraud_reported', axis=1)
y = insurance_df['fraud_reported']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

"""**Model 1: Logistic Regression**"""

# Build a pipeline with SMOTE and Logistic Regression

# Define scoring metrics
scoring = {
    'accuracy': make_scorer(accuracy_score)
}

pipeline = ImbPipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('smote', SMOTE()),
    ('classifier', LogisticRegression(class_weight='balanced'))
])

# Define cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_results = cross_validate(pipeline, x_train, y_train, cv=cv, scoring=scoring)

# Display results
print("Cross-Validation Results:")
for metric in scoring.keys():
    print(f"{metric}: {cv_results[f'test_{metric}'].mean():.4f}")

# Train the model
pipeline.fit(x_train, y_train)

# Make predictions
y_pred = pipeline.predict(x_test)

#Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=['Not Fraud', 'Fraud'], yticklabels=['Not Fraud', 'Fraud'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

"""**Model 2: Random Forest Algorithm**"""

#performing second model using RandomForestClassifier
pipeline = ImbPipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('smote', SMOTE()),
    ('classifier', RandomForestClassifier(class_weight='balanced', random_state=42))
])

# Train the model
pipeline.fit(x_train, y_train)

# Make predictions
y_pred = pipeline.predict(x_test)

#Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=['Not Fraud', 'Fraud'], yticklabels=['Not Fraud', 'Fraud'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

"""**Model 3: XGBoost Classifier**"""

# Count class distribution in training data
class_0_count = sum(y_train == 0)
class_1_count = sum(y_train == 1)

# Calculate the weight
scale_pos_weight = class_0_count / class_1_count
model = XGBClassifier(scale_pos_weight=class_0_count / class_1_count)

#initialize and train the model
xgb_model = XGBClassifier(scale_pos_weight=scale_pos_weight, eval_metric='logloss')
xgb_model.fit(x_train, y_train)


y_probs = xgb_model.predict_proba(x_test)[:, 1]
threshold = 0.3
y_pred = (y_probs >= threshold).astype(int)

#Confustion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=['Not Fraud', 'Fraud'], yticklabels=['Not Fraud', 'Fraud'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

#Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

#perform SHAP analysis to determine variable importance
import shap

# Initialize the SHAP explainer
explainer = shap.TreeExplainer(xgb_model)

# Calculate SHAP values for the test set
shap_values = explainer.shap_values(x_test)

#plot
shap.summary_plot(shap_values, x_test)

"""**Model 4: Ensemble Model**"""

ensemble = VotingClassifier(estimators=[
    ('rf', RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)),
    ('xgb', XGBClassifier(eval_metric='logloss', random_state=42)),
    ('lr', LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42))
], voting='soft')

ensemble.fit(x_train, y_train)

y_probs = ensemble.predict_proba(x_test)[:, 1]
y_pred = (y_probs >= 0.4).astype(int)

#Confustion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=['Not Fraud', 'Fraud'], yticklabels=['Not Fraud', 'Fraud'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

#Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

"""**Model 5: Feedforward Neural Network**"""

# Preprocess: scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Split training into train and validation sets
full_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=10)

# Define the model
class FNNClassifier(nn.Module):
    def __init__(self, input_dim):
        super(FNNClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.fc(x)

model = FNNClassifier(X_train_scaled.shape[1])

# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the model with early stopping
epochs = 100
patience = 10
best_val_loss = float('inf')
patience_counter = 0

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x).squeeze()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_x, val_y in val_loader:
            val_outputs = model(val_x).squeeze()
            val_loss += criterion(val_outputs, val_y).item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch [{epoch+1}/{epochs}] Train Loss: {train_loss/len(train_loader):.4f} | Val Loss: {avg_val_loss:.4f}")

    # Early stopping condition
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

# Load the best model
model.load_state_dict(best_model_state)

# Evaluate on test set
model.eval()
with torch.no_grad():
    y_pred_prob = model(X_test_tensor).squeeze().numpy()
    y_pred = (y_pred_prob > 0.35).astype(int)

#Confustion Matrix
cm = confusion_matrix(y_test.values, y_pred)
sns.heatmap(cm, annot=True, fmt='d', xticklabels=['Not Fraud', 'Fraud'], yticklabels=['Not Fraud', 'Fraud'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

#Classification Report
print("\nClassification Report:")
print(classification_report(y_test.values, y_pred))

"""**Model Comparison**"""

import pandas as pd

# Define the model comparison metrics
models = ['Logistic Regression', 'Random Forest', 'XGBoost', 'Ensemble Model', 'Neural Network']
metrics = {
    'Accuracy': [0.73, 0.77, 0.74, 0.77, 0.75],
    'Precision': [0.69, 0.71, 0.68, 0.71, 0.69],
    'Recall': [0.73, 0.69, 0.68, 0.72, 0.68],
    'F1 Score': [0.70, 0.70, 0.68, 0.71, 0.68]
}

# Create a DataFrame for comparison
model_comparison_df = pd.DataFrame(metrics, index=models)

# Print the table
model_comparison_df

import matplotlib.pyplot as plt
import numpy as np

# Define the model comparison metrics for radar chart
labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
log_reg = [0.73, 0.69, 0.73, 0.70]  # Logistic Regression
rf = [0.77, 0.71, 0.69, 0.70]      # Random Forest
xgb = [0.74, 0.68, 0.68, 0.68]     # XGBoost
ensemble = [0.77, 0.71, 0.72, 0.71]  # Ensemble Model
nn = [0.75, 0.69, 0.68, 0.68]      # Neural Network

# Combine all models
data = np.array([log_reg, rf, xgb, ensemble, nn])

# Number of variables
num_vars = len(labels)

# Compute angle for each axis
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

# Make the plot circular
data = np.concatenate((data, data[:,[0]]), axis=1)
angles += angles[:1]

# Create figure
fig, ax = plt.subplots(figsize=(6, 6), dpi=80, subplot_kw=dict(polar=True))

# Plot each model's data
for i, model in enumerate(['Logistic Regression', 'Random Forest', 'XGBoost', 'Ensemble Model', 'Neural Network']):
    ax.plot(angles, data[i], label=model, linewidth=2, linestyle='solid')

# Fill area
ax.fill(angles, data.T, alpha=0.1)

# Add labels and title
ax.set_yticklabels([])
ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels)
ax.set_title('Model Comparison - Performance Metrics')

# Display the legend
ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.1))

# Show the radar chart
plt.show()

"""**Modeling Summary:**

Five different models were evaluated to identify the best classifier for the data. These models included Logistic Regression, Random Forest, XGBoost, an Ensemble model combining Logistic Regression, Random Forest, and XGBoost, and a Feedforward Neural Network.

The accuracy of all models ranged from 60% to 70%. Logistic Regression showed decent performance, while Random Forest and XGBoost performed well individually.

The ensemble model delivered the best performance with an accuracy of 77%, precision of 71%, recall of 72%, and an F1 score of 71%.

The Feedforward Neural Network model also performed well, yielding results similar to the ensemble model, making it a strong contender for the final recommendation.

Given its flexibility and the ability to tune parameters of the individual models, we recommend using the ensemble model for final deployment.

**Final Recommendation:** Ensemble Model with Logistic Regression, Random Forest and XGBoost can be used for detecting insurance fraud.

**Limitations:**

	1.	The overall accuracy, precision, recall, and F1 scores are in the 70% range, with even lower performance for the fraud class. This indicates that the model is not efficient in correctly identifying fraudulent claims, which is critical for fraud detection systems. The imbalance between fraudulent and non-fraudulent claims likely contributes to this limitation, as the models are more inclined to predict the majority class (non-fraud) due to the class distribution.
	2.	Feature Selection: While some features were selected, the model may still benefit from additional feature engineering, as important patterns could be hidden within less obvious features.

**Future Scope for Improvement:**

	1.	Advanced Feature Engineering: More advanced techniques such as feature scaling, polynomial feature generation, and interaction terms could be explored to capture complex relationships between the features.
	2.	Hyperparameter Optimization: More advanced hyperparameter tuning techniques like Randomized Search or Bayesian Optimization could be used to find the best settings for each model, particularly for more complex models like XGBoost and Neural Networks.
	3.	Ensemble Methods: The current ensemble model combines basic models, but more sophisticated ensemble techniques (e.g., stacking) could be explored to improve the final prediction accuracy.
	4.	Real-time Prediction: The project could evolve into a real-time fraud detection system that dynamically updates the models as new claims come in, making the system more responsive and effective at catching fraudulent claims in real-time.
"""

#import insurance_claims data as df
df = pd.read_csv('insurance_claims.csv')

# List of columns that were label-encoded
label_encoded_cols = ['policy_state', 'incident_state', 'insured_sex', 'insured_education_level',
                      'insured_occupation',
                      'incident_type', 'collision_type', 'incident_severity',
                      'authorities_contacted', 'property_damage',
                      'police_report_available', 'auto_model']

# Create a copy of x_test to decode and export
decoded_x_test = x_test.copy()

# For each categorical column, re-fit a LabelEncoder on insurance_df and decode x_test
for col in label_encoded_cols:
    le = LabelEncoder()
    le.fit(df[col])  # Fit encoder on full dataset
    decoded_x_test[col] = le.inverse_transform(x_test[col])

tableau_df = decoded_x_test.copy()
tableau_df['fraud_reported'] = y_test.values
tableau_df['predicted_fraud'] = y_pred
tableau_df['fraud_probability'] = y_probs


tableau_df['actual_fraud'] = y_test.values
tableau_df.head()

tableau_df.to_csv('tableau_df.csv', index=False)